In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import interpolation as inter
from PIL import Image as im
import pickle
import cv2
from tqdm import tqdm
from scipy.ndimage import rotate
import time
import joblib
from sklearn.metrics import accuracy_score,f1_score,classification_report
import os
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from skimage.exposure import histogram
from matplotlib.pyplot import bar
from scipy.signal import convolve2d
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import LabelBinarizer
from skimage.feature import hog
import time

In [3]:
def show_images(images,titles=None):
    """
    This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    images[0] will be drawn with the title titles[0] if exists.
    """
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        plt.axis('off')
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show() 

In [4]:
def showHist(img):
    """
    This function is used to show the histogram of an image.
    It's an "interface" to matplotlib.axes.Axes.hist() method
    """
    plt.figure()
    imgHist = histogram(img, nbins=256)
    
    bar(imgHist[1].astype(np.uint8), imgHist[0], width=0.8, align='center')

In [5]:
def load_images():
    """
    This function is used to load the images from the fonts-dataset folder.
    """
    images_train = []
    labels_train = []
    filenames = []
    labels = ['Scheherazade New', 'Marhey', 'Lemonada', 'IBM Plex Sans Arabic']
    empty_images_filenames = ["360.jpeg","627.jpeg","853.jpeg"] 
    for i in tqdm(labels):
        for filename in os.listdir(f'fonts-dataset/{i}'):
            img = cv2.imread(f'fonts-dataset/{i}/{filename}', cv2.IMREAD_GRAYSCALE)
            if i == "Lemonada" and filename in empty_images_filenames:
                print(f"{filename} is empty image!")
                continue
            images_train.append(img)
            labels_train.append(i)
            filenames.append(filename)
    return images_train, labels_train, filenames

In [6]:
# labels = ['Scheherazade New', 'Marhey', 'Lemonada', 'IBM Plex Sans Arabic']

# # Load the images
# X_train, y_train_org, filenames = load_images()

# # Change the y_train to numbers
# y_train_org = [labels.index(i) for i in y_train_org]

In [7]:
# len(X_train), len(y_train_org), len(filenames)

## Preprocessing

In [8]:
def find_score(arr, angle):
    """
    Find the score of the skew angle to be used in deskewing the image
    
    Args:
    arr: the image array
    angle: the angle to rotate the image by
    
    Returns:
    hist: the histogram of the image
    score: the score of the skew angle
    """
    
    # mode{‘reflect’, ‘grid-mirror’, ‘constant’, ‘grid-constant’, ‘nearest’, ‘mirror’, ‘grid-wrap’, ‘wrap’}
    data = rotate(arr, angle, reshape=False, order=0, mode='constant', cval=0, prefilter=False)
    hist = np.sum(data, axis=1)
    score = np.sum((hist[1:] - hist[:-1]) ** 2)
    return hist, score

def rotate_image(image, angle):
    """
    Rotates an image by a given angle and fills the remaining pixels with white color.

    Args:
        image: A NumPy array representing the input image.
        angle: The rotation angle in degrees.

    Returns:
        A new NumPy array representing the rotated image.
    """
    # Get image height and width
    height, width = image.shape[:2]

    # Compute the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)

    # Perform the rotation and fill the remaining pixels with white color
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(1, 1, 1))

    return rotated_image

def deskew(binary_img):
    """
    Deskew the image
    
    Args:
    binary_img: the binary image
    
    Returns:
    pix: the deskewed image
    """
    bin_img = (binary_img // 255.0)
    # angles to check for skew angle = 45 degrees and 90 degrees and 180
    angles = np.array ([0 , 45 , 90 , 135 , 180 , 225 , 270 , 315])
    scores = []
    for angle in angles:
        hist, score = find_score(bin_img, angle)
        scores.append(score)

    best_score = max(scores)
    best_angle = angles[scores.index(best_score)]
    # print('Best angle: {}'.format(best_angle))

    # correct skew
    # data = rotate(bin_img, best_angle, reshape=False, order=0)
    data = rotate_image(bin_img, best_angle)
    img = im.fromarray((255 * data).astype("uint8"))

    pix = np.array(img)
    return pix

In [9]:
def preprocess(img):
    """
    Preprocess the image
    
    Args:
    img: the image
    
    Returns:
    img: the preprocessed image
    """
    image_size = 600
    sharpen_kernel = np.array([[0,-1, 0], [-1,5,-1], [0,-1,0]])
    img = cv2.medianBlur(img, 3) # To remove Salt and Pepper noise
    img = cv2.filter2D(img, -1, sharpen_kernel)  # Sharpen the image
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] # Convert the image to binary
    deskewed_img = deskew(img) # Deskew the image
    final_img = cv2.bitwise_not(deskewed_img) if np.mean(deskewed_img) > 127 else deskewed_img # Invert the image if the mean is less than 127 
    final_img = cv2.resize(final_img, (image_size, image_size)) # Resize the image
    return final_img

In [10]:
# # Preprocess the images
# X_train_preprocess = [preprocess(i) for i in tqdm(X_train)]
# X_train_preprocess = np.array(X_train_preprocess)

In [11]:
# # Dump the preprocessed images to a file
# with open('preprocessed_images.pkl', 'wb') as f:
#     pickle.dump(X_train_preprocess, f)

In [12]:
# # Load the preprocessed images
# with open('preprocessed_images.pkl', 'rb') as f:
#     X_train_preprocess = pickle.load(f)

### Segmentation

In [13]:
def save_image(img, folder, title):
    cv2.imwrite(f'./{folder}/{title}.png', img)

In [14]:
def projection(gray_img, axis:str='horizontal'):
    """ 
    Compute the horizontal or the vertical projection of a gray image 
    """
    if axis == 'horizontal':
        projection_bins = np.sum(gray_img, 1).astype('int32')
    elif axis == 'vertical':
        projection_bins = np.sum(gray_img, 0).astype('int32')

    return projection_bins

In [15]:
def projection_segmentation(clean_img, axis, cut=15, min_width=20, min_height=30):
    """Segment the image based on the projection profile

    Args:
        clean_img : Preprocessed image
        axis (str): 'horizontal' or 'vertical'
        cut (int, optional): Gap between the segments. Defaults to 3.
        min_width (int, optional): Width of the segment. Defaults to 5.
        min_height (int, optional): Height of the segment. Defaults to 5.

    Returns:
        _type_: _description_
    """
    segments = []
    start = -1
    cnt = 0

    projection_bins = projection(clean_img, axis)
    for idx, projection_bin in enumerate(projection_bins):

        if projection_bin != 0:
            cnt = 0
        if projection_bin != 0 and start == -1:
            start = idx
        if projection_bin == 0 and start != -1:
            cnt += 1
            if cnt >= cut:
                if axis == 'horizontal':
                    # Line segmentation
                    segment = clean_img[max(start-1, 0):idx, :]
                    # if segment.shape[0] >= min_height:                    
                    segments.append(segment)
                elif axis == 'vertical':
                    # Word segmentation
                    segment = clean_img[:, max(start-1, 0):idx]
                    # if segment.shape[1] >= min_width:
                    segments.append(segment)
                cnt = 0
                start = -1
    
    return segments

#### Line Segmentation

In [16]:
def line_horizontal_projection(image, cut=3): 
    lines = projection_segmentation(image, axis='horizontal', cut=cut)
    return lines

#### Word Segmentation

In [17]:
def word_vertical_projection(line_image, cut=3):
    line_words = projection_segmentation(line_image, axis='vertical', cut=cut)
    line_words.reverse()
    return line_words

In [18]:
def extract_words(img, visual=0):

    lines = line_horizontal_projection(img)
    words = []
    
    for idx, line in enumerate(lines):
        
        if visual:
            # Check for the size of the line to be greater than 30
            # if line.shape[0] > 30:
            save_image(line, 'lines', f'line{idx}')

        line_words = word_vertical_projection(line)
        for w in line_words:
            # if len(words) == 585:
            #     print(idx)
            words.append((w, line))
        # words.extend(line_words)

    # breakpoint()
    if visual:
        for idx, word in enumerate(words):
            # check for the size of the word to be greater than 30
            # print (word[0].shape)
            # if word[0].shape[0] < 100 and word[0].shape[1] > 20 :
            save_image(word[0], 'words', f'word{idx}')
    return words

## Feature Extraction

### HOG

In [19]:
def apply_hog(X_train_preprocess):
    X_train_hog = []
    for i in tqdm(X_train_preprocess):
        X_train_hog.append(hog(i, orientations= 16, pixels_per_cell=(32, 32), cells_per_block=(4, 4), block_norm='L2-Hys'))
    X_train_hog = np.array(X_train_hog)
    return X_train_hog

In [20]:
# X_train_hog = apply_hog(X_train_preprocess)
# print(X_train_hog.shape)

### SIFT

In [21]:
def apply_sift(X_train_preprocess):
    sift = cv2.SIFT_create()

    X_train_sift = []
    for i in tqdm(X_train_preprocess):
        kp, des = sift.detectAndCompute(i, None)
        if des is None:
            # Add a row of zeros to the SIFT descriptors
            des = np.zeros((1, 128))
        des = des.flatten()
        X_train_sift.append(des)
    return X_train_sift
    
# Pad the SIFT descriptors to the maximum length
def pad_sift_descriptors(X_train_sift, fixed_len):
    # Create a generator that yields each padded descriptor on-the-fly
    padded_descriptors = (np.pad(des, (0, max(0, fixed_len - des.shape[0])))[:fixed_len] for des in X_train_sift)

    # Convert the generator to a numpy array
    X_train_sift_np = np.array(list(padded_descriptors))
    return X_train_sift_np

In [22]:
# X_train_sift = apply_sift(X_train_preprocess)

# # Get the maximum keypoint length of the SIFT descriptors
# max_kp = max(len(kp)/128 for kp in X_train_sift)

# # Get the average keypoint length of the SIFT descriptors
# avg_kp = np.mean([len(kp)/128 for kp in X_train_sift])

# # Get the minimum keypoint length of the SIFT descriptors
# min_kp = min(len(kp)/128 for kp in X_train_sift)

# # Print the maximum keypoint length
# print(max_kp)

# # Print the average keypoint length
# print(avg_kp)

# # Print the minimum keypoint length
# print(min_kp)

In [23]:
# fixed_len = 128 * 350
# X_train_sift_np = pad_sift_descriptors(X_train_sift, fixed_len)

# # Print the shape of the numpy array
# print(X_train_sift_np.shape)

In [24]:
# # Concatenate the hog and sift features
# X_train_features = np.concatenate((X_train_hog, X_train_sift_np), axis=1)
# print(X_train_features.shape)

### EDM1

In [25]:
def Laplacian_filter(img):
    laplacian_filter = np.array([
        [-1,-1,-1],
        [-1,8,-1],
        [-1,-1,-1]
    ])
    edge_image = convolve2d(img, laplacian_filter)
    edge_image = np.where(edge_image > 0.5, edge_image, 0)
    edge_image = np.where(edge_image < 0.5, edge_image, 255)

    edge_image = 255 - edge_image
    return edge_image

In [26]:
def get_edm1_matrix(edge_image):
    edm_matrix = np.zeros((3,3))
    # Pad the image with ones
    edge_image = np.pad(edge_image, 1, mode='constant', constant_values=1)
    for i in range(0, edge_image.shape[0]):
        for j in range(0, edge_image.shape[1]):
            if edge_image[i, j] == 0:
                edm_matrix[1,1] += 1
                if edge_image[i, j + 1] == 0:
                    edm_matrix[1,2] += 1
                if edge_image[i + 1, j + 1] == 0:
                    edm_matrix[2,2] += 1
                if edge_image[i + 1, j] == 0:
                    edm_matrix[2,1] += 1
                if edge_image[i + 1, j - 1] == 0:
                    edm_matrix[2,0] += 1
                if edge_image[i, j - 1] == 0:
                    edm_matrix[1,0] += 1
                if edge_image[i - 1, j - 1] == 0:
                    edm_matrix[0,0] += 1
                if edge_image[i - 1, j] == 0:
                    edm_matrix[0,1] += 1
                if edge_image[i - 1, j + 1] == 0:
                    edm_matrix[0,2] += 1
    return edm_matrix

### EDM2

In [27]:
def sort_values(edm1_matrix):
    values = edm1_matrix.flatten()
    edm1_occurrences_sorted = {}
    edm1_occurrences_sorted[values[5]] = [5,3]
    if values[2] not in edm1_occurrences_sorted:
        edm1_occurrences_sorted[values[2]] = [2,6]
    else:
        edm1_occurrences_sorted[values[2]].extend([2,6])
    if values[1] not in edm1_occurrences_sorted:
        edm1_occurrences_sorted[values[1]] = [1,7]
    else:
        edm1_occurrences_sorted[values[1]].extend([1,7])
    if values[0] not in edm1_occurrences_sorted:
        edm1_occurrences_sorted[values[0]] = [0,8]
    else:
        edm1_occurrences_sorted[values[0]].extend([0,8])
    edm1_occurrences_sorted = dict(sorted(edm1_occurrences_sorted.items(), reverse=True))

    lst = []
    for key in edm1_occurrences_sorted:
        lst.extend(edm1_occurrences_sorted[key])
    return lst

In [28]:
def get_first_occurrence(neighboring_indices, edm1_occurrences_sorted_list):
    for idx in edm1_occurrences_sorted_list:
        if idx in neighboring_indices:
            return idx

In [29]:
def get_edm2_matrix(edge_image, edm1_matrix):
    edm2_matrix_flattened = np.zeros(9)
    edm1_occurrences_sorted_list = sort_values(edm1_matrix)
    edm2_matrix_flattened[4] = edm1_matrix[1,1]
    edge_image = np.pad(edge_image, 1, mode='constant', constant_values=1)
    for i in range(0, edge_image.shape[0]):
        for j in range(0, edge_image.shape[1]):
            neighboring_indices = []
            if edge_image[i, j] == 0:
                if edge_image[i, j + 1] == 0:
                    neighboring_indices.append(5)
                if edge_image[i - 1, j + 1] == 0:
                    neighboring_indices.append(2)
                if edge_image[i - 1, j] == 0:
                    neighboring_indices.append(1)
                if edge_image[i - 1, j - 1] == 0:
                    neighboring_indices.append(0)
                if edge_image[i, j - 1] == 0:
                    neighboring_indices.append(3)
                if edge_image[i + 1, j - 1] == 0:
                    neighboring_indices.append(6)
                if edge_image[i + 1, j] == 0:  
                    neighboring_indices.append(7)
                if edge_image[i + 1, j + 1] == 0:
                    neighboring_indices.append(8)

                first_occurrence = get_first_occurrence(neighboring_indices, edm1_occurrences_sorted_list)
                edm2_matrix_flattened[first_occurrence] += 1
    edm2_matrix = edm2_matrix_flattened.reshape(3,3)   
    return edm2_matrix

In [30]:
def apply_edm(X_train_preprocess):
    edge_images = [Laplacian_filter(img) for img in tqdm(X_train_preprocess)]
    
    edm1_matrices = [get_edm1_matrix(edge_img) for edge_img in tqdm(edge_images)]
    edm2_matrices = [get_edm2_matrix(edge_images[i], edm1_matrices[i]) for i in tqdm(range(len(edge_images)))]
    
    edm1_matrices = np.array(edm1_matrices)
    edm2_matrices = np.array(edm2_matrices)

    edm1_matrices = edm1_matrices.reshape(-1,9)
    edm2_matrices = edm2_matrices.reshape(-1,9)
    return edm1_matrices, edm2_matrices

In [31]:
# X_edm1, X_edm2 = apply_edm(X_train_preprocess)
# print(X_edm1.shape)
# print(X_edm2.shape)

In [32]:
# EDM_features = np.concatenate((X_edm1, X_edm2), axis=1)
# print(EDM_features.shape)

In [33]:
def apply_additional_edm_features(X_edm1, X_edm2):
    edges_direction = np.max(X_edm1, axis=1)
    edges_direction = edges_direction.reshape(-1,1)
    homogeneity = np.array([x/np.sum(x) for x in X_edm1])
    pixel_regularity = np.array([x/x[4] for x in X_edm1])
    edges_regularity = np.array([x / x[4] for x in X_edm2])
    return edges_direction, homogeneity, pixel_regularity, edges_regularity

In [34]:
# edges_direction, homogeneity, pixel_regularity, edges_regularity = apply_additional_edm_features(X_edm1, X_edm2)
# print(edges_direction.shape)
# print(homogeneity.shape)
# print(pixel_regularity.shape)
# print(edges_regularity.shape)

In [35]:
# edm_features = np.concatenate((edges_direction, homogeneity, pixel_regularity, edges_regularity), axis=1)

## Pipeline

In [36]:
class PyTorchClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim , learning_rate=0.0002 , epoch=50):
        self.input_dim = input_dim
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.output_dim = output_dim
        self.best_accuracy = -1  # Initialize with a value that will definitely be improved upon
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.model = self.create_model()

    def create_model(self):
        model = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim1),
            nn.ReLU(),
            nn.Linear(self.hidden_dim1, self.hidden_dim2),
            nn.ReLU(),
            nn.Linear(self.hidden_dim2, self.output_dim),
        )
        return model
    
    def fit(self, X_train, X_val, y_train, y_val):
        lb = LabelBinarizer()
        y_train_one_hot = lb.fit_transform(y_train)
        y_val_one_hot = lb.fit_transform(y_val)

        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.LongTensor(y_train_one_hot)

        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.LongTensor(y_val_one_hot)
        
        # Create a dataset
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_val_tensor, y_val_tensor)

        # Create a dataloader
        train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

        # Create a tqdm object
        progress_bar = tqdm(range(self.epoch), desc="Epoch", leave=False)

        for self.epoch in progress_bar:
            total_loss = 0
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                outputs = self.model(X_batch)
                loss = criterion(outputs, torch.max(y_batch, 1)[1])
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            # Calculate accuracy on the validation set
            val_acc = 0
            with torch.no_grad():
                for X_val, y_val in test_loader:
                    outputs = self.model(X_val)
                    _, predicted = torch.max(outputs, 1)
                    val_acc += (predicted == torch.max(y_val, 1)[1]).sum().item()
            accuracy = val_acc / len(y_val_tensor)

            # If the current model has better accuracy, save the model parameters
            if accuracy > self.best_accuracy:
                self.best_accuracy = accuracy
                self.best_model_params = self.model.state_dict()

            # Update the progress bar
            progress_bar.set_postfix({'Loss': f'{total_loss:.4f}', 'Accuracy': f'{self.best_accuracy:.4f}'})
            
        # Save the best model parameters to the model
        self.save_best_model('best_model.pth')

    def predict(self, X):
        X_tensor = torch.FloatTensor(X)
        with torch.no_grad():
            # Set the best_model_params to the model
            predictions = self.model(X_tensor)
        _, predicted = torch.max(predictions, 1)
        return predicted.numpy()

    def save_best_model(self, filepath):
        torch.save(self.best_model_params, filepath)

In [42]:
class Prediction:
    def __init__(self, X_data, X_val, y_labels, y_val, model, preprocess_pipe, labels):
        self.X_train = X_data
        self.X_val = X_val

        self.y_val = [self.labels.index(i) for i in y_val]

        self.labels = labels
        self.y_train =  [self.labels.index(i) for i in y_labels]

        self.preprocess_pipe = preprocess_pipe
        self.model = model

    def preprocess_data(self, X, test = False, X_train_features_pkl_path="X_train_features.pkl", X_train_preprocessed_images_pkl_path="X_train_preprocessed_images.pkl", X_train_hog_sift_pkl_path="X_train_hog_sift.pkl"):
        fixed_len = 128 * 350
        if test:
            X_preprocess = [preprocess(i) for i in tqdm(X)]
            X_preprocess = np.array(X_preprocess)
            X_hog = apply_hog(X_preprocess)
            X_sift = apply_sift(X_preprocess)
            X_sift_padded = pad_sift_descriptors(X_sift, fixed_len)
            X_features = np.concatenate((X_hog, X_sift_padded), axis=1)
            return self.preprocess_pipe.transform(X_features)
        else:
            if os.path.exists(X_train_features_pkl_path):
                with open(X_train_features_pkl_path, 'rb') as f:
                    X_features = pickle.load(f)
                return X_features
            
            if os.path.exists(X_train_preprocessed_images_pkl_path):
                with open(X_train_preprocessed_images_pkl_path, 'rb') as f:
                    X_preprocess = pickle.load(f)
            else:
                X_preprocess = [preprocess(i) for i in tqdm(X)]
                X_preprocess = np.array(X_preprocess)
                with open(X_train_preprocessed_images_pkl_path, 'wb') as f:
                    pickle.dump(X_preprocess, f)
                    
            if os.path.exists(X_train_hog_sift_pkl_path):
                with open(X_train_hog_sift_pkl_path, 'rb') as f:
                    X_features = pickle.load(f)
            else:
                print("Starting Hog features")
                X_hog = apply_hog(X_preprocess)
                print("Done with Hog features")
                
                print("Starting Sift features")
                X_sift = apply_sift(X_preprocess)
                print("Done with Sift features")
                X_sift_padded = pad_sift_descriptors(X_sift, fixed_len)
                
                print("Shapes: ", X_hog.shape, X_sift_padded.shape)

                X_features = np.concatenate((X_hog, X_sift_padded), axis=1)
                with open(X_train_hog_sift_pkl_path, 'wb') as f:
                    pickle.dump(X_features, f)        
                del X_hog, X_sift, X_sift_padded
                    
            print("Starting standardization and PCA")
            X_features_transformed = self.preprocess_pipe.fit_transform(X_features)
            with open("preprocess_pipe.pkl", 'wb') as f:
                pickle.dump(self.preprocess_pipe, f)
            print("Done with standardization and PCA")
            
            print("Shape after PCA: ", X_features_transformed.shape)
            with open(X_train_features_pkl_path, 'wb') as f:
                pickle.dump(X_features_transformed, f)

        # X_edm1, X_edm2 = apply_edm(X_preprocess)
        # edges_direction, homogeneity, pixel_regularity, edges_regularity = apply_additional_edm_features(X_edm1, X_edm2)

        # EDM_features = np.concatenate((edges_direction, homogeneity, pixel_regularity, edges_regularity), axis=1)

        # X_features_transformed = np.concatenate((X_features_transformed, EDM_features), axis=1)

        return X_features_transformed
    
    def train(self):
        X_train_features = self.preprocess_data(self.X_data)

        self.model.fit(X_train_features, self.X_val, self.y_train, self.y_val)
        
        with open("model.pkl", 'wb') as f:
            pickle.dump(self.model, f)

        # Predict the training data
        y_train_pred = self.model.predict(X_train_features)

        # Print the accuracy
        print('Training accuracy: ', accuracy_score(self.y_train, y_train_pred)*100)

        # Print the classification report
        print('Training classification report: ', classification_report(self.y_train, y_train_pred, target_names=self.labels))

    def predict(self, X):
        start_time = time.time()
        X_test_features = self.preprocess_data(X, test=True)
        y_test_pred = self.model.predict(X_test_features)
        end_time = time.time()
        print(f"Time taken to predict: {end_time - start_time}")
        return y_test_pred

In [ ]:
labels = ['Scheherazade New', 'Marhey', 'Lemonada', 'IBM Plex Sans Arabic']
    
# model = LogisticRegression(warm_start=True, solver='saga', penalty='l2', C=0.8, random_state=42)

# preprocess_pipe = Pipeline([
#         ('scaler', StandardScaler()),
#         ('pca', PCA(n_components=0.99)),
#     ])

with open('preprocess_pipe.pkl', 'rb') as f:
    preprocess_pipe = pickle.load(f)

load = False
if load:
    X_data, y_labels, _ = load_images()
    print(len(X_data), len(y_labels))
    print("Done loading data")

    with open('X_data.pkl', 'wb') as f:
        pickle.dump(X_data, f)

    with open('y_labels.pkl', 'wb') as f:
        pickle.dump(y_labels, f)
else:
    with open('X_data.pkl', 'rb') as f:
        X_data = pickle.load(f)

    with open('y_labels.pkl', 'rb') as f:
        y_labels = pickle.load(f)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_data, y_labels, test_size=0.15, random_state=42, stratify=y_labels)

model = PyTorchClassifier(3720, 512, 256, len(labels) , learning_rate=0.00025, epoch=15)

Predictor = Prediction(X_train, X_val, y_train, y_val, model, preprocess_pipe, labels)

Predictor.train()
# Predict the labels of the training set
y_train_pred = Predictor.predict(X_train)

# Predict the labels of the validation set
y_val_pred = Predictor.predict(X_val)

# Print the accuracy of the Random Forest Classifier model on the training set
print(f"Accuracy of Neural Network Classifier model on the training set: {accuracy_score(y_train, y_train_pred)*100}")

# Print the accuracy of the Random Forest Classifier model on the validation set
print(f"Accuracy of Neural Network Classifier model on the validation set: {accuracy_score(y_val, y_val_pred)*100}")

# Print the classification report of the Random Forest Classifier model
print(classification_report(y_train, y_train_pred, target_names=labels))

# Print the classification report of the Random Forest Classifier model
print(classification_report(y_val, y_val_pred, target_names=labels))

## Model Development

### PCA

In [ ]:
# # Create a pipeline for the model which consist of StandardScaler , PCA and the model
# pipe = Pipeline([
#     ('scaler', StandardScaler()),
#     ('pca', PCA(n_components=0.99)),
# ])

# # Fit the pipeline
# pipe.fit(X_train)

In [ ]:
# # Transform the data
# X_train_transformed = pipe.transform(X_train)

# # Print the shape of the transformed data
# print(X_train_transformed.shape)

### Multilayer Perceptron

In [ ]:
# Build a shallow neural network model of 1 hidden layer with 256 neurons and relu activation function
# Create a model of 1 hidden layer with 256 neurons and relu activation function and adam solver and softmax output layer
model = MLPClassifier(hidden_layer_sizes=(256,), activation='relu', solver='adam', verbose=True)

# Fit the model
model.fit(X_train_transformed, y_train)

# Predict the training data
y_train_pred = model.predict(X_train_transformed)

# Predict the testing data
y_test_pred = model.predict(X_test_transformed)

# Print the accuracy of the model
print('Train accuracy: ', accuracy_score(y_train, y_train_pred)*100)

print('Test accuracy: ', accuracy_score(y_test, y_test_pred)*100)

# Print the classification report
print('Train classification report: ', classification_report(y_train, y_train_pred, target_names=labels))

print('Test classification report: ', classification_report(y_test, y_test_pred, target_names=labels))

# Accuracy: 92.75%

### Logistic Regression

In [ ]:
# lin_model = LogisticRegression()

# # Define the hyperparameters
# param_grid = {
#     'C': np.logspace(-4, 4, 30),
#     'penalty': ['l2'],
#     'solver': ['liblinear', 'saga', 'lbfgs'],
#     'warm_start': [True, False]
# }

# # Initialize the RandomizedSearchCV
# random_search = RandomizedSearchCV(lin_model, param_distributions=param_grid, n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)

# # Fit the model
# random_search.fit(X_train_transformed, y_train)
# relevant_columns = ['param_C', 'param_penalty', 'param_solver', 'param_warm_start', 'mean_test_score', 'std_test_score', 'rank_test_score']
# cv_results_df = pd.DataFrame(random_search.cv_results_)[relevant_columns].round(decimals=3).sort_values(by='rank_test_score')
# cv_results_df.head(10)

# # Print the best parameters
# print(random_search.best_params_)

# Best parameters of Logistic Regression: 
# {'warm_start': True, 'solver': 'saga', 'penalty': 'l2', 'C': 0.001}
# {'warm_start': True, 'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.0006723357536499335}

In [ ]:
# Initialize the Logistic Regression model
model = LogisticRegression(warm_start=True, solver='saga', penalty='l2', C=0.8, random_state=42)

# Fit the model
model.fit(X_train_transformed, y_train)

# Predict the test data
y_pred = model.predict(X_test_transformed)

# Predict the train data
y_pred_train = model.predict(X_train_transformed)

# Print the training accuracy
print(f"Training Accuracy: {accuracy_score(y_train, y_pred_train)*100}")

# Print the testing accuracy
print(f"Testing Accuracy: {accuracy_score(y_test, y_pred)*100}")

# Print the f1 score of the training data
print(f"Training F1 Score: {f1_score(y_train, y_pred_train, average='weighted')}")

# Print the f1 score of the testing data
print (f"Testing F1 Score: {f1_score(y_test, y_pred, average='weighted')}")

# Print the classification report of the training data
print(classification_report(y_train, y_pred_train, target_names=labels))

# Print the classification report of the testing data
print(classification_report(y_test, y_pred, target_names=labels))

# Save the model
joblib.dump(model, 'logistic_model.pkl')

# Accuracy: 95.5%

### Support Vector Machine

In [ ]:
# svm = SVC()

# param_dist = {
#     'C': np.logspace(-3, 3, 15), 
#     'kernel': ['poly', 'rbf'], 
#     'degree': [2, 3],
#     'gamma': ['scale', 'auto']
# }

# clf_searched = RandomizedSearchCV(svm, param_dist, n_iter=100, cv=5, random_state=42, n_jobs=-1, verbose=1)

# clf_searched.fit(X_train_transformed, y_train)
# relevant_columns = ['param_C', 'param_kernel', 'param_gamma', 'mean_test_score', 'std_test_score', 'rank_test_score']
# cv_results_df = pd.DataFrame(clf_searched.cv_results_)[relevant_columns].round(decimals=3).sort_values(by='rank_test_score')
# cv_results_df.head(10)

# Best parameters of the SVM
# {C : 7.196857 , kernel : 'rbf', gamma : 'scale'}

In [ ]:
# Initialize the SVM model
svm = SVC(C=7.196857 , kernel='rbf', gamma='scale', random_state=42)

# Fit the SVM on the training data
svm.fit(X_train_transformed, y_train)

# Predict the labels of the test set
y_pred = svm.predict(X_test_transformed)
# Predict the labels of the training set
y_pred_train = svm.predict(X_train_transformed)

# Print the accuracy of the SVM model on the training set
print(f"Accuracy of SVM model on the training set: {accuracy_score(y_train, y_pred_train)*100}")

# Print the accuracy of the SVM model on the test set
print(f"Accuracy of SVM model: {accuracy_score(y_test, y_pred)*100}")

# Print the classification report of the SVM model
print(classification_report(y_train, y_pred_train, target_names=labels))

# Print the classification report of the SVM model
print(classification_report(y_test, y_pred, target_names=labels))

# Save the SVM model as h5 file
joblib.dump(svm, 'svm_model.pkl')

# Accuracy: 91.25%

### Pytorch Neural Network

In [ ]:
class PyTorchClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim , learning_rate=0.0002 , epoch=50):
        self.input_dim = input_dim
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.output_dim = output_dim
        self.best_accuracy = -1  # Initialize with a value that will definitely be improved upon
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.model = self.create_model()

    def create_model(self):
        model = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim1),
            nn.ReLU(),
            nn.Linear(self.hidden_dim1, self.hidden_dim2),
            nn.ReLU(),
            nn.Linear(self.hidden_dim2, self.output_dim),
        )
        return model
    
    def fit(self, X_train, X_val, y_train, y_val):
        lb = LabelBinarizer()
        y_train_one_hot = lb.fit_transform(y_train)
        y_val_one_hot = lb.fit_transform(y_val)

        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.LongTensor(y_train_one_hot)

        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.LongTensor(y_val_one_hot)
        
        # Create a dataset
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_val_tensor, y_val_tensor)

        # Create a dataloader
        train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

        # Create a tqdm object
        progress_bar = tqdm(range(self.epoch), desc="Epoch", leave=False)

        for self.epoch in progress_bar:
            total_loss = 0
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                outputs = self.model(X_batch)
                loss = criterion(outputs, torch.max(y_batch, 1)[1])
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            # Calculate accuracy on the validation set
            val_acc = 0
            with torch.no_grad():
                for X_val, y_val in test_loader:
                    outputs = self.model(X_val)
                    _, predicted = torch.max(outputs, 1)
                    val_acc += (predicted == torch.max(y_val, 1)[1]).sum().item()
            accuracy = val_acc / len(y_val_tensor)

            # If the current model has better accuracy, save the model parameters
            if accuracy > self.best_accuracy:
                self.best_accuracy = accuracy
                self.best_model_params = self.model.state_dict()

            # Update the progress bar
            progress_bar.set_postfix({'Loss': f'{total_loss:.4f}', 'Accuracy': f'{self.best_accuracy:.4f}'})
            
        # Save the best model parameters to the model
        self.save_best_model('best_model.pth')

    def predict(self, X):
        X_tensor = torch.FloatTensor(X)
        with torch.no_grad():
            # Set the best_model_params to the model
            predictions = self.model(X_tensor)
        _, predicted = torch.max(predictions, 1)
        return predicted.numpy()

    def save_best_model(self, filepath):
        torch.save(self.best_model_params, filepath)

In [ ]:
# Initialize the PyTorchClassifier
pytorch_clf = PyTorchClassifier(X_train_transformed.shape[1], 512, 256, len(labels) , learning_rate=0.00025, epoch=15)

# Fit the PyTorchClassifier
pytorch_clf.fit(X_train_transformed, X_test_transformed, y_train, y_test)

# Predict the labels of the test set
y_pred = pytorch_clf.predict(X_test_transformed)

# Predict the labels of the training set
y_pred_train = pytorch_clf.predict(X_train_transformed)

# Print the accuracy of the Random Forest Classifier model on the training set
print(f"Accuracy of Neural Network Classifier model on the training set: {accuracy_score(y_train, y_pred_train)*100}")

# Print the accuracy of the Random Forest Classifier model on the test set
print(f"Accuracy of Neural Network Classifier model on the test set: {accuracy_score(y_test, y_pred)*100}")

# Print the classification report of the Random Forest Classifier model
print(classification_report(y_train, y_pred_train, target_names=labels))

# Print the classification report of the Random Forest Classifier model
print(classification_report(y_test, y_pred, target_names=labels))

# Accuracy:  96.75%

In [ ]:
# # Build a neural networ consist of input layer of size 400, hidden layer of size 256 , hidden layer of size 128 and output layer of size 4 pytorch
# # Convert the data to tensors
# X_train_tensor = torch.tensor(X_train_transformed, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.long)
# X_test_tensor = torch.tensor(X_test_transformed, dtype=torch.float32)
# y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# # Print the shape of the tensors
# print(X_train_tensor.shape, y_train_tensor.shape, X_test_tensor.shape, y_test_tensor.shape)

# # Create a dataset
# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# # Create a dataloader
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# # Define the neural network
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(X_train_tensor.shape[1], 512)
#         self.fc2 = nn.Linear(512, 256)
#         self.fc3 = nn.Linear(256, 4)
        
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x
    
# # Create the model
# model = Net()

# # Define the loss function
# criterion = nn.CrossEntropyLoss()

# # Define the optimizer
# optimizer = optim.Adam(model.parameters(), lr=0.0002)

# best_score = 0.0

# # Train the model
# for epoch in range(30):
#     model.train()
#     running_loss = 0.0
#     pbar = tqdm(enumerate(train_loader), total=len(train_loader))
#     for i, (X, y) in pbar:
#         optimizer.zero_grad()
#         output = model(X)
#         loss = criterion(output, y)
#         loss.backward()
#         optimizer.step()

#         # update running loss value
#         running_loss += loss.item()

#         # update the progress bar with the average loss
#         pbar.set_description(f"Epoch {epoch+1} Loss: {running_loss/(i+1):.4f}")

#     # reset running loss for next epoch
#     running_loss = 0.0

#     # Evaluate the model
#     model.eval()

#     # Get the predictions
#     y_pred = []
#     y_true = []
#     for X, y in test_loader:
#         output = model(X)
#         y_pred.extend(torch.argmax(output, 1).tolist())
#         y_true.extend(y.tolist())

#     # Calculate the accuracy
#     accuracy = accuracy_score(y_true, y_pred)*100
#     print('Accuracy: ', accuracy)

#     # If the accuracy is better than the best score so far, save the model
#     if accuracy > best_score:
#         best_score = accuracy
#         torch.save(model.state_dict(), 'best_model.pth')

# # Print the classification report
# print(classification_report(y_true, y_pred, target_names=labels))

### Stacking Classifier

In [ ]:
# Stacking the models together to get the best model
from sklearn.ensemble import StackingClassifier

# Define the base models
# Stacking the models together to get the best model

# Define the base models
estimators = [
    # ('logistic', LogisticRegression(warm_start=True, solver='saga', penalty='l2', C=0.8, random_state=42)),
    ('neural_net1', PyTorchClassifier(X_train_transformed.shape[1], 512, 512, len(labels) , learning_rate=0.0002, epoch=10)),
    ('logistic',  LogisticRegression(warm_start=True, solver='saga', penalty='l2', C=0.8, random_state=42))
    
]
meta_model = LogisticRegression()

# Initialize the stacking classifier
stacking_clf = StackingClassifier(estimators=estimators, final_estimator=meta_model, cv=5)

# Fit the stacking classifier
stacking_clf.fit(X_train_transformed , y_train)

# Predict the labels of the test set
y_pred = stacking_clf.predict(X_test_transformed)

# Predict the labels of the training set
y_pred_train = stacking_clf.predict(X_train_transformed)

# Print the accuracy of the Stacking Classifier model on the training set
print(f"Accuracy of Stacking Classifier model on the training set: {accuracy_score(y_train, y_pred_train)*100}")

# Print the accuracy of the Stacking Classifier model on the test set
print(f"Accuracy of Stacking Classifier model: {accuracy_score(y_test, y_pred)*100}")

# Print the classification report of the Stacking Classifier model
print(classification_report(y_train, y_pred_train, target_names=labels))

# Print the classification report of the Stacking Classifier model
print(classification_report(y_test, y_pred, target_names=labels))

# Accuracy: 96%